In [1]:
import pandas as pd
import numpy as np

In [2]:
import dlp
import os
os.chdir('/project/work')

ModuleNotFoundError: No module named 'dlp'

In [3]:
from preprocess import *
from Data_Load import *
from parameters import *

ModuleNotFoundError: No module named 'model.Preprocessing'

from pycaret.regression import *

## 1. 기본 데이터 
#### (1시간 단위의 교통 카드 이력 데이터, 정류장별 위/경도 좌표 포함)

In [4]:
# 현재 환경에 마운트되어 있는 데이터셋 키값, 경로를 출력한다
for i, dataset_key in enumerate(dlp.dataset_paths.keys()):
    print(" * 데이터셋 KEY : %s\t DATAPATH: %s"%(dataset_key.rjust(10), dlp.dataset_paths[dataset_key]))

 * 데이터셋 KEY :       mybi	 DATAPATH: /datasets/DTS2021000012


In [5]:
base_data = pd.read_csv(dlp.dataset_paths['mybi']+"/data/base_data.csv")
target_col = "totalcnt"

In [6]:
base_data.dtypes

Unnamed: 0      int64
stop_nm        object
latitude      float64
longitude     float64
stop_id         int64
transdate      object
totalcnt        int64
normalcnt       int64
studentcnt      int64
childcnt        int64
dtype: object

In [7]:
#날짜 type변환
base_data['transdate'] = pd.to_datetime(base_data['transdate'])

In [8]:
base_year = base_data["transdate"].dt.strftime("%Y")[0]
city = "울산"

In [9]:
path = os.getcwd() + '/data/api_data'

## 2. 외부 데이터 수집

In [10]:
from utils.createFolder import *

In [11]:
createFolder(os.getcwd() + '/data')
createFolder(os.getcwd() + '/data/api_data')

In [12]:
# 휴일 정보 수집
Load_Holiday_Data(holiday_params_dict, save_tf = True, save_path = path)

In [13]:
# 날씨 정보 수집
Load_Weather_Data(weather_params_dict, save_tf = True, save_path = path)

n_rows : 900, total_count : 8064, max_page = 9


In [14]:
# 미세먼지 경보 정보 수집
Load_Particulate_Matter_Data(pm_params_dict, save_tf = True, save_path = path)

n_rows : 1000, total_count : 304, max_page = 1


In [15]:
# 상권 정보 수집
Load_Trading_Data(trading_params_dict,
                  google_key = googleKey,
                  select_region = city,
                  save_tf = True, 
                  save_path = path)

n_rows : 100, total_count : 36517, max_page = 366


In [16]:
# 병원 정보 수집
Load_Hospital_Data(hospital_params_dict, save_tf = True, save_path = path)

n_rows : 1000, total_count : 1406, max_page = 2


In [17]:
# 학교(초중고) 정보 수집
Load_School_Data(school_params_dict, select_region = city, save_tf = True, save_path = path)

n_rows : 1000, total_count : 500, max_page = 1


In [18]:
# 대학교 정보 수집
Load_University_Data(university_params_dict,
                     google_key = googleKey,
                     select_region = city,
                     save_tf = True, save_path = path)

In [19]:
# 행사 정보 수집
Load_Event_Data(event_params_dict,
                start_year = base_year,
                select_region = city,
                save_tf = True,
                save_path = path)

n_rows : 1000, total_count : 6396, max_page = 7


In [20]:
# 축제 정보 수집
Load_Festival_Data(festival_params_dict,
                   start_year = base_year,
                   select_region = city,
                   save_tf = True,
                   save_path = path)

n_rows : 1000, total_count : 1026, max_page = 2


## 3. 외부 데이터 Load

In [22]:
# 휴일 정보 Load
holiday_data = pd.read_csv(path + "/holiday_data.csv")
# 날씨 정보 Load
weather_data = pd.read_csv(path + "/weather_data.csv")
# 미세먼지 경보 정보 Load
pm_data = pd.read_csv(path + "/pm_data.csv")
# 상권 정보 load
trading_area_data = pd.read_csv(path + "/trading_data.csv")
# 병원 정보 Load
hospital_data = pd.read_csv(path + "/hospital_data.csv")
# 초중고등학교 정보 Load
school_data = pd.read_csv(path + "/school_data.csv")
# 대학교 정보 Load
school_data = pd.read_csv(path + "/university_data.csv")
# 행사 정보 Load
event_data = pd.read_csv(path + "/event_data.csv")
# 축제 정보 Load
festival_data = pd.read_csv(path + "/festival_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
preprocessing = preprocess(data = base_data,
                           date_col = "transdate", 
                           target_cols = target_col, 
                           stop_id_col = "stop_id", 
                           holiday_data = holiday_data,
                           weather_data = weather_data,
                           pm_data = pm_data,
                           trading_area_data = trading_area_data,
                           hospital_data = hospital_data,
                           school_data = school_data,
                           university_data = university_data,
                           event_data = event_data,
                           festival_data = festival_data,
                           num_cores = 12)

In [17]:
ml_data = preprocessing.run()

1. 결측치 Impute ... Finished ((563300, 14))
2. 시계열 변수 생성 : Finished ((563300, 66))
3. 시간적 특성 변수 추가 (특일, 날씨, 미세먼지 경보) ... Finished ((563300, 74))
4. 공간적 특성 정보 추가 (상권정보, 학교정보, 병원정보) ... Finished ((563300, 175))
5. 시공간적 특성 정보 추가 (행사정보, 축제정보) ... Finished ((563300, 177))


In [18]:
del base_data

## 4. 모델 학습

In [19]:
categorical_feature = ["dayofweek", "hour", "ntl_holi", "holi", "seq_holi"]

In [20]:
numerical_feature = [col for col in ml_data.columns if col not in categorical_feature + [target_col]]

#### 데이터 Setup

In [21]:
reg_experiment = setup(ml_data, 
                       target = "totalcnt", 
                       categorical_features = categorical_feature,
                       numeric_features = numerical_feature,
                       ignore_features = ["month", "weekofyear", "longitude", "latitude", "studentcnt", "childcnt", "normalcnt"],
                       train_size = 0.7,
                       n_jobs = 14,
                       session_id = 12345,
#                        log_experiment = True,
#                        log_plots = True,
#                        log_profile = True,
#                        log_data = True,
#                        profile = True,
                       silent = True,
                       experiment_name="Passenger_Demand")

,Description,Value
0,session_id,12345
1,Target,totalcnt
2,Original Data,"(563300, 173)"
3,Missing Values,True
4,Numeric Features,160
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(394309, 189)"


#### 초도 모델링

In [ ]:
%%time
base_models = compare_models(include = ["lightgbm", "rf", "gbr", "lr", "ridge", "lasso", "en", "knn"])

IntProgress(value=0, description='Processing: ', max=44)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:18:31
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.5870,8.2300,2.8686,0.7677,0.5520,0.5649,7.2730


#### 모델 선택 및 생성

In [ ]:
%%time
lightgbm = create_model('lightgbm')

#### 모델 튜닝

In [ ]:
%%time
tuned_lightgbm_optuna_tpe = tune_model(lightgbm, 
                                       optimize = "RMSE", 
                                       search_library = "optuna", 
                                       search_algorithm = "tpe", 
                                       fold = 10, 
                                       return_tuner = True, 
                                       n_iter = 10)